**To make grading efficient and smooth we ask you to follow the rules:**

<ol>
<li>A single notebook file (without archiving) per group should be submitted via BB.  
<li>Name of the notebook should be your group number, e.g., 01.ipynb or 21.ipynb, not <strike>group21.ipynb</strike> or <strike>group_1.ipynb</strike>.    
<li>Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE".
<li>Delete those cells that you inserted for your own debuging/testing purposes.
</ol>

**Not following these rules might result in point deduction.**

<div class="alert alert-block alert-info">
<b>Note:</b>
Be aware that some code or markdown cells are read-only to prevent you from changing them and complicating grading for graders.  
</div>

**Overwrite this text** with a brief description who contributed to which parts

**Delete this cell** unless you would like to leave a message for graders or give a feedback about the assigment. Writing it here is better than in the BlackBoard submission textbox.

---

# Vectorial Word Representations

## Background
Representing words as dense vectors over a finite-dimensional space was one of the recent breakthroughs in Natural Language Processing. Vectorial representations allow space-efficient, informationally rich storage of words that adequately captures their semantic content and enables numerical computation on them. Word vectors are the standard input representation for machine learning architectures for language processing. Even though new methods for constructing such representations emerge frequently, the original set of published papers remain a de facto point of reference as well as a good starting point. For this assignment, you will be asked to implement a small-scale variant of one such paper, namely [Global Word Vectors for Word Representation](https://nlp.stanford.edu/pubs/glove.pdf) ("the GloVe paper").

Much of the code and data pre-processing has already been done for you. Additionally, notes on the paper will appear throughout the notebook to guide you along the code. It is, however, important to read and understand the paper, its terminology and the theory behind it before attempting to go through with the assignment. Some of the tasks will also require addressing the paper directly.

**-------------------------------------------------------------------------------------------------------------**

There are 2 types of tasks in this assignment: 
- coding tasks --- 10 tasks worth 1 point each --- asking you to write code following specifications provided; Most of the tasks come with test cases for sanity-check. Still, if something is not clear, <ins>do ask questions to lab teachers</ins>.
- interpretation questions --- 5 questions worth 1 point each --- asking you to interpret the data or the results of the model

You are greatly encouraged to add comments to your code describing what particular lines of code do (in general, a great habit to have in your coding life), as well as self-check regularly by printing your tensors and their shapes making sure they look adequate.

## Corpus Statistics

The paper's proposed model, GloVe, aims to densely represent words in a way that captures the global corpus statistics. 

The construction it encodes is the word __co-occurrence matrix__. A co-occurrence matrix is a simplistic data structure that counts the number of times each word has appeared within the context of every other word. The definition of a context varies; usually, context is implied to be a fixed-length span (that may or may not be allowed to escape sentence boundaries) around a word. 

For instance, in the sentence below and for a context length of 2, the word <span style="color:pink">__Earth__</span> occurs in the context of <span style="color:lightgreen">made</span> (1), <span style="color:lightgreen">on</span> (1), <span style="color:lightgreen">as</span> (1), <span style="color:lightgreen">an</span> (1).

> "He struck most of the friends he had <span style="color:lightgreen">made on</span> <span style="color:pink">__Earth__</span> <span style="color:lightgreen">as an</span> eccentric"

Similarly, the word <span style="color:pink">__friends__</span> occurs in the context of <span style="color:lightgreen">of</span> (1), <span style="color:lightgreen">the</span> (1), <span style="color:lightgreen">he</span> (1), <span style="color:lightgreen">had</span> (1).

> "He struck most <span style="color:lightgreen">of the</span> <span style="color:pink">__friends__</span> <span style="color:lightgreen">he had</span> made on Earth as an eccentric"

An alternative definition of a context would be, for instance, the variable-length windows spanned by a full sentence.

Contexts may be summed across sentences or entire corpora; the summed context of <span style="color:pink">he</span> in the example sentence is: <span style="color:lightgreen">struck</span> (1), <span style="color:lightgreen">most</span> (1), <span style="color:lightgreen">the</span> (1), <span style="color:lightgreen">friends</span> (1), <span style="color:lightgreen">had</span> (1), <span style="color:lightgreen">made</span> (1).



For the purposes of this assignment, we have prepared a co-occurrence matrix over a minimally processed version of the Harry Potter books.

(A few interpretation tasks in this assignment presuppose some minimal level of familiarity with the Harry Potter books/films. If no one in your group is familiar with Harry Potter, you might find the [fandom page](https://harrypotter.fandom.com/wiki/Main_Page) useful or the [synopsis sections](https://en.wikipedia.org/wiki/Harry_Potter_and_the_Philosopher%27s_Stone) of the corresponding wiki pages.

The pickle file contains three items:
1. `vocab`: a dictionary mapping words to unique ids, containing $N$ unique words
2. `contexts`: a dictionary mapping words to their contexts, where contexts are themselves dicts from words to integers that show the number of co-occurrences between these words.
    E.g. `{"portrait": {"harry": 103, "said": 97, ...}, ...}` meaning that the word "harry" has appeared in the context of the word "portrait" 103 times, etc.
3. `X`: a torch LongTensor ${X}$ of size $N \times N$, where ${X}[i,j]$ denotes the number of times the word with id $j$ has appeared in the context of the word with id $i$

Extremely common or uncommon words (i.e. words with too few or too many global occurrences) have been filtered out for practical reasons.

In [ ]:
import pickle
import torch
from torch import FloatTensor, LongTensor
from typing import Dict, Callable, List
# torch.set_printoptions(precision=8) #to increase precision of printing floats 

In [ ]:
with open("output.p", "rb") as f:
    vocab, contexts, X = pickle.load(f)
print(len(vocab))

Let's inspect the summed context of the word 'portrait'.

In [ ]:
sorted([(item, value) for item, value in contexts["portrait"].items()], key=lambda x: x[1], reverse=True)

How about the word 'ghost'?

In [ ]:
sorted([(item, value) for item, value in contexts["ghost"].items()], key=lambda x: x[1], reverse=True)

The co-occurrence matrix of a very large corpus should give a meaningful summary of how a word is used in general. A single row of that matrix is already a __word vector__ of size $N$. However such vectors are extremely sparse, and for large corpora the size of $N$ will become unwieldy. We will follow along the paper in designing an algorithm that can compress the word vectors while retaining most of their informational content. 

<div class="alert alert-block alert-info">
<b>Note:</b>
For the resulting vectors to actually be informative, the source corpus should have a size of at least a few billion words; on the contrary, our corpus enumerates merely a million words, so we can't expect our results to be as great.
</div>

### Sparsity and Stability

Our matrix $X$ is very sparse; most of its elements are zero.

**Coding 1.1**: Find what the ratio of non-zero elements is.

_Hint_: The function `non_zero_ratio` should return a `float` rather than a `FloatTensor`. Remember `.item()`.

In [ ]:
def non_zero_ratio(sparse_matrix: LongTensor) -> float:
    NotImplemented
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
assert 0.1 < non_zero_ratio(X) < 0.2

We will soon need to perform division and find the logarithm of ${X}$. Neither of the two operations are well-defined for $0$. That's why for further processing we want to have a matrix without any zero elements. 

**Coding 1.2**: Change the matrix's datatype to a `torch.float` and add a small constant to it (e.g. $0.1$) to ensure numerical stability while maintaining sparsity. The obtained matrix will be used in the remaining sections (not the original one).

In [ ]:
X1 = NotImplemented
# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
assert X1.dtype == torch.float32
assert non_zero_ratio(X1) == 1
assert X1[0,0] == 0.1  # note that tensor(True) is evaluated as True in Boolean context
assert X1[0,73] == 1.1

<div class="alert alert-block alert-warning">
<b>Show the completed code to your teacher if you have any doubts</b>
</div>

### From co-occurrence counts to probabilities
From the paper: 
> Let the matrix of word-word co-occurrence counts be denoted by $X$, whose entries $X_{ij}$ tabulate the number of times word $j$ occurs in the context of word $i$.  Let $X_i$= $\sum_{k} X_{ik}$ be the number of times any word appears in the context of word $i$. Finally, let $P_{ij} = P(j  | i) =  X_{ij}/X_i$ be the probability that word $j$ appear in the context of word $i$. 

**Coding 2**: Complete the function `to_probabilities` that accepts a co-occurrence matrix and returns the probability matrix $P$. 

_Hint_: Remember broadcasting and `torch.sum()`.

In [ ]:
def to_probabilities(count_matrix: FloatTensor) -> FloatTensor:
    NotImplemented
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
P = to_probabilities(X1) # note that we use X1 not X here

In [ ]:
assert P.shape == torch.Size([5359, 5359])
assert P[0,0].isclose(FloatTensor([0.00012439]))
assert P[0,73].isclose(FloatTensor([0.00136833]))

<div class="alert alert-block alert-warning">
<b>Show the completed code to your teacher if you have any doubts</b>
</div>

### Probing words

From the paper:
> Consider two words $i$ and $j$ that exhibit a particular aspect of interest. The relationship of these words can be examined by studying the ratio of their co-occurrence probabilities with various probe words, $k$.  For words $k$ related to $i$ but not $j$, we expect the ratio $P_{ik}/P_{jk}$ will be large.  Similarly, for words $k$ related to $j$ but not $i$, the ratio should be small. For words $k$ that are either related to both $i$ and $j$, or to neither, the ratio should be close to one.

**Coding 3.1**: Complete the function `query` that accepts two words $w_i$ and $w_j$, a vocab $V$ and a probability matrix ${P}$, maps each word to its corresponding index and returns the probability $P(j  |  i)$. If such probability is impossible to be computed for input words, return float 0. probability.  

In [ ]:
def query(word_i: str, word_j: str, vocab: Dict[str, int], prob_matrix: FloatTensor) -> float:
    # 
    NotImplemented
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
assert round(query('harry', 'potter', vocab, P), 5) == 0.00394
assert round(query('potter', 'potter', vocab, P), 6) == 5e-6
assert round(query('hello', 'there', vocab, P)) == 0

**Coding 3.2**: Then, complete the function `probe` that accepts three words $w_i$, $w_j$ and $w_k$, a vocab $V$ and a probability matrix ${P}$, calls `query` and returns the ratio $P(k |  i) / P(k  |  j)$.

In [ ]:
def probe(word_i: str, word_j: str, word_k: str, vocab: Dict[str, int], prob_matrix: FloatTensor) -> float:
    NotImplemented
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
assert round(probe('harry', 'potter', 'stone', vocab, P), 4) == 1.4417

<div class="alert alert-block alert-warning">
<b>Show the completed code to your teacher if you have any doubts</b>
</div>

Let's probe a few words and examine whether the authors' claim holds even for our (tiny) corpus. Feel free to add your own word triplets and experiment.

In [ ]:
print("tea", "wand", "spell", probe("tea", "wand", "spell", vocab, P))
print("tea", "wand", "cup", probe("tea", "wand", "cup", vocab, P))
print()

# YOUR CODE HERE
raise NotImplementedError()

**Interpretation 1**: Give a brief interpretation of the results you got. Do they correspond to your expectations? Why or why not?

*Hint*: When do we expect the ratio value to be high, low or close to 1? Refer to the GloVe paper for guidance.

YOUR ANSWER HERE

What would happen if we tried probing out-of-domain words? Use the words that the authors report in the paper as discriminative for "ice" and "steam". Make your code to clealry print the details.

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

**Interpretation 2**: Give an interpretation of the results you got. Do they match what the authors report in the paper? Why or why not?

YOUR ANSWER HERE

## Dense Vectors

Now, we would like to convert these long sparse vectors into short dense ones. 

The conversion should be such that the probability ratios we inspected earlier may still be reconstructed via some (for now, unknown) operation $F$ on the dense vectors.

To restrict the search space over potential functions, the authors impose a number of constraints they think $F$ should satisfy:
1. > While $F$ could be taken to be a complicated function parameterized by, e.g., a neural network, doing so would obfuscate the linear structure we are trying to capture. $F$ should be dot-product based.
2. > The distinction between a word and a context word is arbitrary and we are free to exchange the two roles. To do so consistently, we must not only exchange $w \leftrightarrow \tilde{w}$ but also $X \leftrightarrow X^T$.
3. > It should be well-defined for all values in $X$.

Given these three constraints, each word $i$ in our vocabulary is represented by four vectors:
1. A vector $w_i \in \mathbb{R}^D$
2. A bias $b_i \in \mathbb{R}$
3. A context vector $\tilde{w}_i \in \mathbb{R}^D$
4. A context bias $\tilde{b}_i \in \mathbb{R}$

and $F: \mathbb{R}^D \times \mathbb{R} \times \mathbb{R}^D \times \mathbb{R} \to \mathbb{R}$ is defined as:

$F(w_i, \tilde{w}_k, b_i, \tilde{b}_k) = w_i^T\tilde{w}_k + b_i + \tilde{b}_k$.

Or equivalently the least squares error $J$ is minimized, where:

$J = \sum_{i,j=1}^{V} f(X_{ij})(w_{i}^T\tilde{w}_j + b_i + \tilde{b}_j - log(X_{ij}))^2$

with $f$ being a weighting function, defined as 

$f: \mathbb{R} \to \mathbb{R} = \begin{cases}
    (x/x_{max})^\alpha, & \text{if $x<x_{max}$}\\
    1, & \text{otherwise}.
  \end{cases}$

### Weighting Function

Let's start with the last part. 

**Coding 4**: Complete the weighting function `weight_fn` which accepts a co-occurrence matrix ${X}$, a maximum value $x_{max}$ and a fractional power $alpha$, and returns the weighted co-occurrence matrix $f({X})$.

Then, compute $\text{X_weighted}$, the matrix ${X}$ after weighting, using the paper's suggested parameters. 

_Hint_: Note that $f$ is defined pointwise, so our weighting function should also be pointwise.

In [ ]:
def weight_fn(X: FloatTensor, x_max: int, alpha: float) -> FloatTensor:
    NotImplemented
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
X_weighted = weight_fn(X1, x_max=100, alpha=3/4)

In [ ]:
assert X_weighted.shape == X1.shape
assert X_weighted[0,0].isclose(FloatTensor([0.00562341]))
assert X_weighted[3,3955].isclose(FloatTensor([1]))

Try to get an understanding of how the weighting affects different co-occurrence values (high and low). Think of some word pairs with high and low co-occurrence and look them up in $X$ and in $\text{X_weighted}$ to get a better idea. 

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

<div class="alert alert-block alert-warning">
<b>Show the completed code to your teacher if you have any doubts</b>
</div>

### Loss Function

Next step is to write the loss function. 

We can write it as a pointwise function, apply it iteratively over each pair of words and then sum the result; that's however extremely inefficient. 

Inspecting the formulation of $J$, it is fairly straight-forward to see that it can be immediately implemented using matrix-matrix operations, as:

$J = \sum_{i,j=1}^{V}f(\mathbf{X})\cdot(W\tilde{W}^T + b + \tilde{b}^T - log(X))^2$,

where $W$, $\tilde{W}$ are the $N \times D$ matrices containing the $D$-dimensional vectors of all our $N$ vocabulary words, and $b$, $\tilde{b}$ are the $N \times 1$ matrices containing the $1$-dimensional biases of our words.

**Coding 5**: Complete `loss_fn`, a function that accepts a weighted co-occurrence matrix $f({X})$, the word vectors and biases $W$, $\tilde{W}$, $b$, $\tilde{b}$ and the co-occurrence matrix ${X}$, and computes $J$. 

In [ ]:
def loss_fn(
    X_weighted: FloatTensor, 
    W: FloatTensor, 
    W_context: FloatTensor, 
    B: FloatTensor, 
    B_context: FloatTensor, 
    X: FloatTensor
) -> FloatTensor:
    NotImplemented
    # YOUR CODE HERE
    raise NotImplementedError()

Let's make sure that we are on a right track. For this we calculate the loss function with toy input: matrices are of size $2 \times 2$ while bias vectors of size $2 \times 1$. You can verify the answer manually and with your implementation of `loss_fn`.

In [ ]:
toy_X_weighted = torch.FloatTensor([[.5,1],[.2,.1]])
toy_X1 = torch.FloatTensor([[2,1],[1,5]])
toy_W1 = torch.FloatTensor([[1,2],[1,0]]) # for W
toy_W2 = torch.FloatTensor([[0,1],[1,2]]) # for W~
toy_b1 = torch.FloatTensor([[0],[2]]) # for b
toy_b2 = torch.FloatTensor([[2],[1]]) # for b~
assert loss_fn(toy_X_weighted, toy_W1, toy_W2, toy_b1, toy_b2, toy_X1).isclose(FloatTensor([45.239116]))


<div class="alert alert-block alert-warning">
<b>Show the completed code to your teacher if you have any doubts</b>
</div>

### GloVe

We have the normalized co-occurrence matrix ${X}$, the weighting function $f$, and the loss function $J$ that implements $F$.

What we need now is a mapping from words (or word ids) to unique, parametric and trainable vectors. 

Torch provides this abstraction in the form of [Embedding layers](https://pytorch.org/docs/stable/generated/torch.nn.Embedding.html?highlight=embedding#torch.nn.Embedding). Each such layer may be viewed as a stand-alone network that can be optimized using the standard procedure we have already seen. It is recommended to read about Embedding class  

We will utilize the `nn.Module` class to contain all our embedding layers and streamline their joint optimization.
The container class will be responsible for a few things:

1. **Coding 6.1**: Wrapping the embedding layers:
    1. A vector embedding that maps words to $w \in \mathbb{R}^D$
    2. A context vector embedding that maps words to $w_c \in \mathbb{R}^D$
    3. A bias embedding that maps words to $b \in \mathbb{R}^1$
    4. A context bias embedding that maps words to $b_c \in \mathbb{R}^1$
2. **Coding 6.2**: Implementing `forward`, a function that accepts a weighted co-occurrence matrix $f(X)$, the co-occurrence matrix $X$, then finds the embeddings of all words and finally calls `loss_fn` as defined above.
3. **Coding 7**: Implementing `get_vectors`, a function that receives no input and produces the word vectors and context word vectors of all words, adds them together and returns the result, in accordance with the paper:
> ...With this in mind, we choose to use the sum $W + \tilde{W}$ as our word vectors.

Complete the network class following the above specifications.  

In [ ]:
class GloVe(torch.nn.Module):
    def __init__(self, vocab: Dict[str, int], vector_dim: int=30, device: str="cpu", seed: int=0) -> None:
        super(GloVe, self).__init__()
        self.device = device
        self.vocab_len = len(vocab)
        torch.manual_seed(seed) #random initialization of w, wc, b, bc is fixed by the seed 
        self.w = NotImplemented
        self.wc = NotImplemented
        self.b = NotImplemented
        self.bc = NotImplemented        
        # YOUR CODE HERE
        raise NotImplementedError()
        
    def forward(self, X_weighted: FloatTensor, X: FloatTensor) -> FloatTensor:
        embedding_input = torch.arange(self.vocab_len).to(self.device)
        W = self.w(embedding_input)
        # YOUR CODE HERE
        raise NotImplementedError()
        return NotImplemented
    
    def get_vectors(self) -> FloatTensor:
        embedding_input = torch.arange(self.vocab_len).to(self.device)
        # YOUR CODE HERE
        raise NotImplementedError()
        return NotImplemented

<div class="alert alert-block alert-warning">
<b>Show the completed code to your teacher if you have any doubts</b>
</div>

### Training

Everything is in place; now we may begin optimizing our embedding layers (and in doing so, the vectors they assign). 

**Coding 8.1**: Instantiate the network class you just defined using $D = 30$. Then instantiate an `Adam` optimizer with a learning rate of 0.05 and train your network for 300 epochs.

When writing the training script, remember that your network's forward pass is __already__ computing the loss.

In [ ]:
network = NotImplemented
opt = NotImplemented

# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
num_epochs = 300
losses = [] # collect losses for each epoch here
for i in range(num_epochs):
    NotImplemented # loss computation (optionally print it out), remember to use X1 counts not X
    NotImplemented # gradient computation
    NotImplemented # back-propagation
    NotImplemented # gradient reset
    
    # YOUR CODE HERE
    raise NotImplementedError()

Note that if you want to re-run the training process from scratch, remember that you also need to redefine `network` otherwise your training will continue from the last epoch's training state.  
<font color="red">**Don't clear the output of the above cell!**</font>

In [ ]:
assert len(losses) == 300

**Coding 8.2**: Plot the losses (x axis for epoch number and y axis for loss) and examine the learning curve. Ask yourself, is its shape what you would expect it to be?  
<font color="red">**Don't clear the output of the below cell!**</font>

In [ ]:
# Your plotting here

# YOUR CODE HERE
raise NotImplementedError()

<div class="alert alert-block alert-warning">
<b>Show the completed code to your teacher if you have any doubts</b>
</div>

### Validation (Similarity)

Curious to see what this network has learned? Let's perform a simple validation experiment. 

We will check which words the model considers the most similar to other words. To that end, we need a notion of __similarity__. One of the most common measures of similarity in high dimensional vector spaces is the cosine similarity. 

The cosine similarity of two vectors $\vec{a}, \vec{b}$ is given as:
$$sim(\vec{a}, \vec{b}) = \frac{\vec{a}\cdot \vec{b}}{|\vec{a}|_2 \cdot |\vec{b}|_2}$$

where $|\vec{x}|_2$ is the $L_2$-norm of the $\vec{x}$.

The function `similarity` below accepts two words, a vocabulary and the network's output vectors, and computes the similarity between these two words. For an outside-vocabulary word similarity is 0.

In [ ]:
def similarity(word_i: str, word_j: str, vocab: Dict[str, int], vectors: FloatTensor) -> float:
    if not(word_i in vocab and word_i in vocab): return 0.
    i = vocab[word_i]
    j = vocab[word_j] 
    v_i = vectors[i] / torch.linalg.vector_norm(vectors[i])  # a/|a|
    v_j = vectors[j] / torch.linalg.vector_norm(vectors[j])  # b/|b|
    sim = torch.mm(v_i.view(1, -1), v_j.view(-1, 1)).item()
    # sim = sim = torch.dot(v_i, v_j)
    return sim

Let's check out some examples. Consider the word pairs below and, optionally, add your own word pairs if it helps to support your answer:

In [ ]:
word_vectors = network.get_vectors().detach()

for pair in [
    ("cruciatus", "imperius"), 
    ("avada", "kedavra"), 
    ("hogwarts", "school"), 
    ("goblin", "hagrid"), 
    ("giant", "hagrid"),
    # YOUR CODE HERE
    raise NotImplementedError()
]:
    print(f"Similarity between '{pair[0]}' and '{pair[1]}' is: {similarity(pair[0], pair[1], vocab, word_vectors)}")

**Interpretation 3**: Give a brief interpretation of the results. Do the scores correspond well to your perceived similarity of these word pairs?

YOUR ANSWER HERE

To obtain the similarities of one word against all other words in the corpus, we may rewrite the above equation as:
$$sim(\vec{w}, \mathbf{C}) = \frac{\vec{w}\cdot \mathbf{C}}{|\vec{w}|_2 \cdot |\mathbf{C}|_2}$$

**Coding 9**: Using `similarity` as a reference, write `similarities`, which accepts one word, a vocabulary and the network's output vectors and computes the similarity between the word and the entire corpus. If a word is out of vocabulary, it should return a matrix of 0 similarities. 

_Hint_: $\mathbf{C} \in \mathbb{R}^{N, D}$, $\vec{w} \in \mathbb{R}^{1, D}$, $sim(\vec{w}, \mathbf{C}) \in \mathbb{R}^{1, N}$

In [ ]:
def similarities(word_i: str, vocab: Dict[str, int], vectors: FloatTensor) -> FloatTensor:
    NotImplemented
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
assert similarities('harry', vocab, word_vectors).shape == torch.Size([1, 5359])
assert similarities('cow', vocab, word_vectors).shape == torch.Size([1, 5359])

<div class="alert alert-block alert-warning">
<b>Show the completed code to your teacher if you have any doubts</b>
</div>

Now we can manipulate the word vectors to find out what the corpus-wide most similar words to a query word are!

In [ ]:
def most_similar(word_i: str, vocab: Dict[str, int], vectors: FloatTensor, k: int) -> List[str]:
    """ Returns a list of k words that are most similar to word_i
        The list excludes word_i itself
    """
    sims = similarities(word_i, vocab, vectors)
    _, topi = sims.topk(dim=-1, k=k)
    topi = topi.view(-1).cpu().numpy().tolist()
    inv = {v: i for i, v in vocab.items()}
    return [inv[i] for i in topi if inv[i] != word_i]

In [ ]:
for word in [
    "forbidden", "myrtle", "gryffindor", "wand", "quidditch", "marauder",
    "horcrux", "phoenix", "triwizard", "screaming", "letter"
]:
    print("Most similar words to '{}': {}".format(word, most_similar(word, vocab, word_vectors, 6)))

**Interpretation 4**: Interpret the results.
- Do these most similar words make sense (are they actually similar to the query words)? 
- Are there any patterns you can see in the "errors" (the words that you woudn't consider actually similar to the query word in general, in everyday life)? 
- Would you say that the model captures similarity, relatedness, both or neither?
- Any other observations are welcome.

Illustrate your answers with examples from your model's output.

YOUR ANSWER HERE

Overall it's quite impressive; we managed to encode a meaningful portion of the corpus statistics in only $30$ numbers per word! 
(A compression ratio of 99.4%)

<div class="alert alert-block alert-info">
<b>Note:</b> The word vectors obtained by this process are (to a small extent) random, due to the random initialization of the embedding layers. If you are unhappy with your results, you can repeat the experiment a few times or try to toy around with the hyper-parameters (the smoothing factor of ${X}$, $x_{max}$, $\alpha$, the number of epochs and the dimensionality of the vector space).
</div>

Word vectors, however, contain way more information than just word co-occurrence statistics. Hold tight until the next assignment, where we will see how word vectors may be used to infer information spanning entire phrases and sentences.

### Validation (Word Analogies)

From the paper:
> The word analogy task consists of questions like "$a$ is to $b$ as is $c$ to $?$" To correctly answer this question, we must find the word $d$ such that $w_d \approx w_b - w_a + w_c$ according to the cosine similarity.

**Coding 10**: Write your own function that performs the word analogy task.

_Hint_: Take a look at the code a few cells back. Most of what you need is already there.

In [ ]:
def analogy(
    word_a: str, word_b: str, word_c: str, vocab: Dict[str, int], vectors: FloatTensor, k: int
) -> List[str]:
    """ Return a list of k words whose vectors are most similar to the solution vector of the analogy.
        word_a, word_b, and word_c are never returned as a part of the list 
    """
    NotImplemented
    # YOUR CODE HERE
    raise NotImplementedError()

<div class="alert alert-block alert-warning">
<b>Show the completed code to your teacher if you have any doubts</b>
</div>

Some example triplets to test your analogies on:

In [ ]:
triplets = [("padma", "parvati", "fred"),
            ("avada", "kedavra", "expecto"),
            ("dungeon", "slytherin", "tower"),
            ("scabbers", "ron", "hedwig"),
            ("ron", "molly", "draco"),
            ("durmstrang", "viktor", "beauxbatons"),
            ("snape", "potions", "trelawney"),
            ("harry", "seeker", "ron")
           ]

for a, b, c in triplets:
    print("'{}' is to '{}' as '{}' is to {}".format(a, b, c, analogy(a, b, c, vocab, word_vectors, 6)))

Some minimal emergent intelligence :) *(hopefully..)*. 🧙‍♀️

**Interpretation 5**: Interpret the results. 
- Did the model manage to guess the correct answers to the analogies (taking the first word in the output to be the model's "guess")? 
- Are the correct answers present in the top K words? 
- Do you see any patterns in the cases when the model didn't solve the task correctly? In other words, when the model's guess was wrong, can you suggest why the model guessed what it guessed?

**ANSWER**: Note that the above output mismatches the answer from last year because of randomness involved in training the vectors.

<font color="red">Three of the analogy tasks were solved correctly: "avada kedavra" - "expecto patronum", "ron molly" - "draco narcissa", "snape potions" - "trelawney divination".</font> 

<font color="red">In one other case the correct answer was present in the top K: "dungeon slytherin" - "tower gryffindor/ravenclaw" (both houses had their common room in a tower).</font> 

<font color="red">In some other cases the top K words were topically related to the prompts, however, didn't include the correct answer to the analogy: "harry-seeker" had the top K words related to quidditch, "durmstrang viktor" had the top K words (roughly) related to the Triwizard Tournament.</font> 

<font color="red">The "padma parvati" analogy would have "george" as the correct answer (the twin of Fred) but one of the top K answers was "ron", who is also one of Fred's brothers.</font> 

<font color="red">In the "scabbers ron" analogy all the top K words were very far off, with no apparent logic in the model's predictions.</font> 

### Optional
If you are done, you can continue experimenting in order to understand the system's behaviour better. For example: how does training and hyperparameter choice affect the model's performance?
Repeat the training using your own hyperparameters (vector space dimensionality, optimizer parameters, the number of training epochs, different random seed, etc.). 

During the training loop, print the qualitative benchmarks every few epochs. Do they keep improving? Is there any disadvantage to exhaustively training until convergence?